In [0]:
configs = {
"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret" : '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/keyid/oauth2/token"}

dbutils.fs.mount(
    source = "abfss://landing-zone-2@ecomadlssid.dfs.core.windows.net",
    mount_point = "/mnt/ecomdata",
    extra_configs = configs)


True

In [0]:
%fs ls "/mnt/ecomdata/"

path,name,size,modificationTime
dbfs:/mnt/ecomdata/buyers-raw-2/,buyers-raw-2/,0,1747737693000
dbfs:/mnt/ecomdata/countries-raw-2/,countries-raw-2/,0,1747737712000
dbfs:/mnt/ecomdata/sellers-raw-2/,sellers-raw-2/,0,1747737704000
dbfs:/mnt/ecomdata/users-raw-2/,users-raw-2/,0,1747737687000


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName("Ecom data pipeline").getOrCreate()

In [0]:
spark

In [0]:
userDF = spark.read.format("parquet").option("header","true").option("inferSchema","true").load("/mnt/ecomdata/users-raw-2")


In [0]:
userDF.show()

+--------------------+----+-----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----------------+-----------+
|      identifierhash|type|    country|language|socialnbfollowers|socialnbfollows|socialproductsliked|productslisted|productssold|productspassrate|productswished|productsbought|gender|civilitygenderid|civilitytitle|hasanyapp|hasandroidapp|hasiosapp|hasprofilepicture|dayssincelastlogin|seniority|seniorityasmonths|seniorityasyears|countrycode|
+--------------------+----+-----------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+----------------

In [0]:
userDF.printSchema()

root
 |-- identifierhash: string (nullable = true)
 |-- type: string (nullable = true)
 |-- country: string (nullable = true)
 |-- language: string (nullable = true)
 |-- socialnbfollowers: string (nullable = true)
 |-- socialnbfollows: string (nullable = true)
 |-- socialproductsliked: string (nullable = true)
 |-- productslisted: string (nullable = true)
 |-- productssold: string (nullable = true)
 |-- productspassrate: string (nullable = true)
 |-- productswished: string (nullable = true)
 |-- productsbought: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- civilitygenderid: string (nullable = true)
 |-- civilitytitle: string (nullable = true)
 |-- hasanyapp: string (nullable = true)
 |-- hasandroidapp: string (nullable = true)
 |-- hasiosapp: string (nullable = true)
 |-- hasprofilepicture: string (nullable = true)
 |-- dayssincelastlogin: string (nullable = true)
 |-- seniority: string (nullable = true)
 |-- seniorityasmonths: string (nullable = true)
 |-- senio

In [0]:
userDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/bronze/users")

In [0]:
buyersDF = spark.read.format("parquet").option("header","true").option("inferSchema","true").load("/mnt/ecomdata/buyers-raw-2")


In [0]:
buyersDF.show(5)

+-----------+------+---------+-------------+------------+----------+---------------+-------------+-----------------+--------------------+----------------------+------------------+-------------------------+---------------------+-------------------+-------------------+------------------+----------------------+----------------------+---------------------+------------------+------------------+-----------------+---------------------+---------------------+--------------------+---------------+------------------+-------------+-------------+----------------+----------------+
|    country|buyers|topbuyers|topbuyerratio|femalebuyers|malebuyers|topfemalebuyers|topmalebuyers|femalebuyersratio|topfemalebuyersratio|boughtperwishlistratio|boughtperlikeratio|topboughtperwishlistratio|topboughtperlikeratio|totalproductsbought|totalproductswished|totalproductsliked|toptotalproductsbought|toptotalproductswished|toptotalproductsliked|meanproductsbought|meanproductswished|meanproductsliked|topmeanproductsbo

In [0]:
buyersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/bronze/buyers")

In [0]:
countriesDF = spark.read.format("parquet").option("header","true").option("inferSchema","true").load("/mnt/ecomdata/countries-raw-2")
sellersDF = spark.read.format("parquet").option("header","true").option("inferSchema","true").load("/mnt/ecomdata/sellers-raw-2")


In [0]:
countriesDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/bronze/countries")
sellersDF.write.format("delta").mode("overwrite").save("/mnt/delta/tables/bronze/sellers")

In [0]:
%fs ls "/mnt/delta/tables/bronze/buyers"

path,name,size,modificationTime
dbfs:/mnt/delta/tables/bronze/buyers/_delta_log/,_delta_log/,0,1747744991000
dbfs:/mnt/delta/tables/bronze/buyers/part-00000-ab0c0b93-eac5-401b-bc57-3c4926b7a1c5.c000.snappy.parquet,part-00000-ab0c0b93-eac5-401b-bc57-3c4926b7a1c5.c000.snappy.parquet,15176,1747744992000
